In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.au,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.au,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.au,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [3]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [4]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
X_valid=X_test[100:]
y_valid=y_test[100:]
X_test=X_test[:100]
y_test=y_test[:100]

In [8]:
import os
import numpy as np
import time
import copy
from glob import glob

import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.autograd import Variable
import random


In [9]:
n_classes = 10



In [10]:
type(X_test)

numpy.ndarray

In [118]:
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1=nn.Linear(X_train.shape[2],256)
        self.fc2=nn.Linear(256,128)
        self.fc3=nn.Linear(128,64)
        self.fc4=nn.Linear(64,n_classes)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=F.softmax(self.fc4(x))
        return x
model_scratch = Net()
print (model_scratch)



Net(
  (fc1): Linear(in_features=26, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [119]:
criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.001)
#optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.001, momentum=0.9)

In [120]:
def train(n_epochs, loaders, model, optimizer, criterion,  save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()

        for i in range(len(loaders['X_train'][1])):
            # move to GPU
            
            data, target = loaders['X_train'], loaders['y_train']
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # calculate the batch loss
            loss = criterion(output, target.long())
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss = train_loss + (1 / (i + 1)) * (loss.data - train_loss)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for m in range(len(loaders['X_valid'][1])):


            data, target = loaders['X_valid'], loaders['y_valid']
            ## update the average validation loss
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float())
            # calculate the batch loss
            loss = criterion(output, target.long())
            # update average validation loss 
            valid_loss = valid_loss + (1 / (m + 1)) * (loss.data - valid_loss)

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [121]:

X_train=Variable(torch.from_numpy(X_train))
X_valid=Variable(torch.from_numpy(X_valid))
y_train=Variable(torch.from_numpy(y_train))
y_valid=Variable(torch.from_numpy(y_valid))


TypeError: expected np.ndarray (got Tensor)

In [122]:
X_train=X_train.reshape(80,-1,26)
y_train=y_train.reshape(80,-1)

In [123]:
n_epochs = 20
loaders_scratch = {'X_train':X_train,'X_valid':X_valid,'y_train':y_train,'y_valid':y_valid}
model_scratch = train(n_epochs, loaders_scratch, model_scratch, optimizer_scratch,criterion_scratch,save_path= 'model_scratch.pt')


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Epoch: 1 	Training Loss: 2.302332 	Validation Loss: 2.289289
Validation loss decreased (inf --> 2.289289).  Saving model ...
Epoch: 2 	Training Loss: 2.301255 	Validation Loss: 2.237999
Validation loss decreased (2.289289 --> 2.237999).  Saving model ...
Epoch: 3 	Training Loss: 2.297318 	Validation Loss: 2.161524
Validation loss decreased (2.237999 --> 2.161524).  Saving model ...
Epoch: 4 	Training Loss: 2.287112 	Validation Loss: 2.143605
Validation loss decreased (2.161524 --> 2.143605).  Saving model ...
Epoch: 5 	Training Loss: 2.272378 	Validation Loss: 2.127277
Validation loss decreased (2.143605 --> 2.127277).  Saving model ...
Epoch: 6 	Training Loss: 2.257269 	Validation Loss: 2.125877
Validation loss decreased (2.127277 --> 2.125877).  Saving model ...
Epoch: 7 	Training Loss: 2.246855 	Validation Loss: 2.134461
Epoch: 8 	Training Loss: 2.240268 	Validation Loss: 2.125043
Validation loss decreased (2.125877 --> 2.125043).  Saving model ...
Epoch: 9 	Training Loss: 2.235586 

In [73]:
X_train.long

<function Tensor.long>